In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test_esrb_df = pd.read_csv("/kaggle/input/video-games-rating-by-esrb/test_esrb.csv")


In [ ]:
vidgme_esrb_df = pd.read_csv("/kaggle/input/video-games-rating-by-esrb/Video_games_esrb_rating.csv")
other_cols = ["title", "console", "esrb_rating"]

def clean_df(df):
    # Filter out games with no descriptors
    df = df[df["no_descriptors"] == 0]
    df = df.drop("no_descriptors", axis=1)
    return df

vidgme_esrb_df = clean_df(vidgme_esrb_df)

In [ ]:
esrb_percentages_df = vidgme_esrb_df.drop(other_cols, axis=1).sum() / len(vidgme_esrb_df)
esrb_percentages_df

In [ ]:
vidgme_esrb_df.pivot_table(index=["esrb_rating", "console"])

### Interesting tidbit: Shovel Knight is the only game with drugs and alcohol with a rating of 'Everyone' in this dataset 
### https://www.esrb.org/ratings/36588/Shovel+Knight%3A+Treasure+Trove/
### https://www.gonintendo.com/stories/350467-shovel-knight-treasure-trove-rated-by-the-esrb
### In the official rating it says Use of alcohol only
### A bit strange?

In [ ]:
pd.set_option('display.max_columns', None)
evryone_esrb_df = vidgme_esrb_df.groupby(["esrb_rating"]).get_group("E")
evryone_esrb_df[evryone_esrb_df["use_of_drugs_and_alcohol"] == 1]

In [ ]:
# .style.background_gradient(cmap ='viridis')
vidgme_esrb_df.drop(other_cols, axis=1).corr().style.background_gradient(cmap ='coolwarm')

In [ ]:
def get_x_and_y(df):    
    x = df.drop(other_cols + ["esrb_rating"], axis=1)
    y = df["esrb_rating"]
    return x, y

x, y = get_x_and_y(vidgme_esrb_df)

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc = svc.fit(x, y)
svc.score(x, y)

In [ ]:
x_test, y_test = get_x_and_y(clean_df(test_esrb_df))
svc.score(x_test, y_test)

### Support vector classifier scored 0.8442 on test set